<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inner" data-toc-modified-id="Inner-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inner</a></span></li><li><span><a href="#Left" data-toc-modified-id="Left-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Left</a></span></li><li><span><a href="#Filter-with-left-semi" data-toc-modified-id="Filter-with-left-semi-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Filter with left semi</a></span></li><li><span><a href="#What's-not-in-left--with-anti-join" data-toc-modified-id="What's-not-in-left--with-anti-join-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>What's not in left  with anti join</a></span></li></ul></div>

In [27]:
spark.createDataFrame([(1,),(2,),(3,)], schema="col int").show()

+---+
|col|
+---+
|  1|
|  2|
|  3|
+---+



In [30]:
left  = spark.createDataFrame([(1,"a"),(1,"a"),(2,"b"),(3,"c")], schema="col int, left string")
right = spark.createDataFrame([(3,"c"),(3,"c"),(4,"d"),(5,"e")], schema="col int, right string")

# Inner

In [31]:
left.join(right, on = "col", how = "inner").show()

+---+----+-----+
|col|left|right|
+---+----+-----+
|  3|   c|    c|
|  3|   c|    c|
+---+----+-----+



# Left

In [32]:
left.join(right, on = "col", how = "left").show()

+---+----+-----+
|col|left|right|
+---+----+-----+
|  1|   a| null|
|  1|   a| null|
|  2|   b| null|
|  3|   c|    c|
|  3|   c|    c|
+---+----+-----+



# Filter with left semi

Note no duplicates

In [25]:
left.join(right, on = "col", how = "left_semi").show()

+---+----+
|col|left|
+---+----+
|  3|   c|
+---+----+



In [29]:
left.join(right, on = "col", how = "left_semi").explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(LeftSemi,Buffer(col))
:- LogicalRDD [col#67, left#68], false
+- LogicalRDD [col#71, right#72], false

== Analyzed Logical Plan ==
col: int, left: string
Project [col#67, left#68]
+- Join LeftSemi, (col#67 = col#71)
   :- LogicalRDD [col#67, left#68], false
   +- LogicalRDD [col#71, right#72], false

== Optimized Logical Plan ==
Join LeftSemi, (col#67 = col#71)
:- Filter isnotnull(col#67)
:  +- LogicalRDD [col#67, left#68], false
+- Project [col#71]
   +- Filter isnotnull(col#71)
      +- LogicalRDD [col#71, right#72], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [col#67], [col#71], LeftSemi
   :- Sort [col#67 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(col#67, 200), ENSURE_REQUIREMENTS, [id=#920]
   :     +- Filter isnotnull(col#67)
   :        +- Scan ExistingRDD[col#67,left#68]
   +- Sort [col#71 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(col#71, 200), ENSURE_REQUIREMENTS

# What's not in left  with anti join

In [26]:
left.join(right, on = "col", how = "left_anti").show()

+---+----+
|col|left|
+---+----+
|  1|   a|
|  2|   b|
+---+----+

